###  <div align="right"> Name: Vipada Siripatanadilok No. 6020422001,Prasit Chulanutrakul No. 6120412007 </div> 
###  <div align="right"> ชื่อ:วิภาดา สิริพัฒนดิลก No. 6020422001, ประสิทธิ์ จุฬานุตรกุล No. 6120412007 </div> 

# Library Loading

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
sns.set()
random_state = 77

import pickle

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

from xgboost import XGBClassifier

from lightgbm import LGBMClassifier


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## 6. Hyperparameter Tuning
ทำการ Tune Hyperparameter โดยกำหนด CV = 10 ด้วยโมเดลในรายการข้างล่าง เพื่อหา Parameter เหมาะสุด
    - k-nearest neighbors
    - Naive Bayes
    - Decision Tree
    - Support Vector Machine
    - Logistic Regression
    - Random Forest
    - Ada Boost
    - Neural Network

In [ ]:
# List of Models
model_list = {
    'k-nearest neighbors':KNeighborsClassifier(),
#     'Naive Bayes':GaussianNB(),
#     'Decision Tree':DecisionTreeClassifier(random_state=random_state),
#     'Support Vector Machine':svm.SVC(random_state=random_state),
#     'Logistic Regression':LogisticRegression(random_state=random_state),
#     'Random Forest':RandomForestClassifier(random_state=random_state),
#     'Ada Boost':AdaBoostClassifier(random_state=random_state),
#     'Neural Network':MLPClassifier(random_state=random_state)
}

model_param_grid = {
    'k-nearest neighbors':{
                    'Model__n_neighbors':(3,5,7,9)
                    },
    'Naive Bayes':{
                    'Model__var_smoothing':(1e-9,0.5,1)
                    },
    'Decision Tree':{
                    'Model__criterion':('gini', 'entropy'), 
                    'Model__max_depth':(None, 1, 3, 5, 7), 
                    'Model__min_samples_split':(2, 3, 4, 5),
                    'Model__min_samples_leaf':(1, 2, 3, 4, 5),
                    'Model__max_features':('auto', 'sqrt', 'log2')
                    },
    'Support Vector Machine':{
                    'Model__kernel':('linear', 'rbf', 'poly'), 
                    'Model__C':(1, 5, 10),
                    'Model__gamma':('scale', 'auto')
                    },
    'Logistic Regression':{
                    'Model__penalty':('l1', 'l2'),
                    'Model__fit_intercept':(True, False),
                    'Model__solver':('liblinear',),
                    'Model__C':(0.5, 1, 2, 5, 10, 100)
                    },
    'Random Forest':{
                    'Model__n_estimators':(100, 150, 200),
                    'Model__criterion':('gini', 'entropy'), 
                    'Model__max_depth':(None, 3, 5, 7), 
                    'Model__min_samples_split':(2, 3, 4),
                    'Model__min_samples_leaf':(1, 2, 3),
                    },
    'Ada Boost':{
                    'Model__n_estimators':(50, 100, 150, 200),
                    'Model__learning_rate':(0.5, 1, 2),
                    'Model__algorithm':('SAMME', 'SAMME.R' )
                    },
    'Neural Network':{
                    'Model__hidden_layer_sizes':((100), (50, 50), (20, 20, 20)),
                    'Model__activation':('logistic', 'tanh', 'relu'),
                    'Model__solver':('lbfgs', 'sgd', 'adam'),
                    'Model__batch_size':('auto', 15, 30),
                    'Model__learning_rate':('constant', 'invscaling', 'adaptive'),
                    'Model__learning_rate_init':(0.001, 0.01, 0.1)
                     }
}

In [ ]:
# Create Pipelines
pipeline_model_list = {}
for model in model_list:
    print(f'Created pipeline for {model}')
    pipeline_model_list[model] =  create_pipeline(model_list[model])

In [ ]:
gridsearch_model_list = {}
for model in pipeline_model_list:
    gridsearch_model_list[model] = GridSearchCV(pipeline_model_list[model], model_param_grid[model], 
                                                    cv =10 , error_score='raise', verbose=5, n_jobs=-1)
    gridsearch_model_list[model].fit(X_train, y_train)
    print(f'Finished Grid Search for {model}')

In [ ]:
summary_result = []
for model in gridsearch_model_list:
    print(f'Validation Result for best parameters of {model}')
    #print(gridsearch_model_list[model].cv_results_)
    #print(gridsearch_model_list[model].best_estimator_)
    print('\nBest CV Accuracy: ',gridsearch_model_list[model].best_score_)
    print('Best Parameters: ', gridsearch_model_list[model].best_params_)

    y_pred = gridsearch_model_list[model].predict(X_test)
    summary_result.append(evaluation(model, y_test, y_pred))
    
    print('\n-- Confusion Martix --')
    print(confusion_matrix(y_test,y_pred))
    print('\n-- Classification Report --')
    print(classification_report(y_test, y_pred))
    print('\n')
    
summary_result_grid_search_df = pd.DataFrame(summary_result)

In [ ]:
summary_result = []
for model in gridsearch_model_list:
    print(f'ฺฺฺ\n----- Best parameters of {model} -----')
    print('Best Parameters: ', gridsearch_model_list[model].best_params_)
    print('Best Parameters: ', gridsearch_model_list[model].best_estimator_)

### Summary Result of Predicting Test Set (Using the Best Hyperparameters from Grid Search)
จะได้ผลลัพท์ของการทดสอบโมเดลด้วย Test Set กับโมเดลที่ Train ด้วย Parameter เหมาะสุดจากการ Grid Search ดังตารางสรุปด้านล่าง

In [ ]:
col = ['MODEL','ACCURACY','PRECISION_1','PRECISION_2','PRECISION_3','RECALL_1','RECALL_2','RECALL_3','F1_1','F1_2','F1_3']
summary_result_grid_search_df

### สรุป